In [1]:
import time
import requests
import csv
import os
from tqdm import tqdm

In [3]:
def fetch_10k_links(cik, filing_type="10-K", retries=3):
    """Fetch all 10-K form links for a given CIK from the SEC."""
    cik_str = str(cik).zfill(10)  # Ensure CIK is zero-padded to 10 digits
    url = f"https://data.sec.gov/submissions/CIK{cik_str}.json"
    
    headers = {
        "User-Agent": "Xploremate nainaditya29@gmail.com"  
    }

    attempt = 0
    while attempt < retries:
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                filings = data['filings']['recent']
                accession_numbers = filings['accessionNumber']
                forms = filings['form']
                filing_dates = filings['filingDate']
                primary_documents = filings['primaryDocument']
                links = []
                
                # Extract links for the specified filing type (10-K)
                for i, form in enumerate(forms):
                    if form == filing_type:
                        accession_no = accession_numbers[i].replace("-", "")
                        filing_link = f"https://www.sec.gov/Archives/edgar/data/{cik_str}/{accession_no}/{primary_documents[i]}"
                        links.append({
                            'Filing Date': filing_dates[i],
                            'Filing Link': filing_link
                        })
                return links
            
            else:
                print(f"Failed to fetch data for CIK: {cik} with status code {response.status_code}")
                attempt += 1
                time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for CIK: {cik}. Error: {e}")
            attempt += 1
            time.sleep(1)

    return []

In [5]:
def load_company_data(input_csv):
    """Load company data from a CSV file and return as a list of dictionaries."""
    companies = []
    with open(input_csv, mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            companies.append({
                'title': row['Company Name'],
                'cik_str': row['CIK'],
                'ticker': row['Ticker']
            })
    return companies

In [7]:
def save_10k_links_to_csv(companies, output_file="us_company_10k_links(2).csv", failed_file="failed_companies.csv", checkpoint_file="checkpoint.csv"):
    """Fetch 10-K links for multiple companies and save them in a CSV file."""
    fieldnames = ['CIK', 'Ticker', 'Company Name', 'Filing Date', '10-K Link']

    # Load checkpoint to resume from last successful company
    processed_ciks = set()
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as checkpoint:
            processed_ciks = set(line.strip() for line in checkpoint)

    with open(output_file, mode='a', newline='') as csvfile, \
         open(failed_file, mode='a', newline='') as failedfile, \
         open(checkpoint_file, mode='a') as checkpoint:
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        failed_writer = csv.writer(failedfile)
        
        # Write headers only if the file is new
        if os.stat(output_file).st_size == 0:
            writer.writeheader()
        if os.stat(failed_file).st_size == 0:
            failed_writer.writerow(['CIK', 'Ticker', 'Company Name'])
        
        for company in tqdm(companies, desc="Processing Companies"):
            cik = company['cik_str']
            if cik in processed_ciks:
                continue  # Skip companies that are already processed
            
            name, ticker = company['title'], company['ticker']
            print(f"Fetching 10-K links for {name} (CIK: {cik})...")
            links = fetch_10k_links(cik)
            
            if links:
                for link in links:
                    writer.writerow({
                        'CIK': cik,
                        'Ticker': ticker,
                        'Company Name': name,
                        'Filing Date': link['Filing Date'],
                        '10-K Link': link['Filing Link']
                    })
            else:
                print(f"No 10-K filings found for {name}.")
                failed_writer.writerow([cik, ticker, name])
            
            # Mark the company as processed in the checkpoint
            checkpoint.write(f"{cik}\n")
            
            time.sleep(0.1)  # Delay between requests to avoid throttling

In [9]:
if __name__ == "__main__":
    # Load the list of companies from a CSV file (input_csv should contain CIK, Ticker, Company Name)
    input_csv = "us_company_ticks.csv"  # Replace this with your input CSV file
    companies = load_company_data(input_csv)
    
    # Fetch and save the 10-K links to a CSV file
    save_10k_links_to_csv(companies)

Processing Companies:   0%|                           | 0/10197 [00:00<?, ?it/s]

Fetching 10-K links for Apple Inc. (CIK: 320193)...


Processing Companies:   0%|                 | 1/10197 [00:00<2:44:28,  1.03it/s]

Fetching 10-K links for MICROSOFT CORP (CIK: 789019)...


Processing Companies:   0%|                 | 2/10197 [00:01<2:39:42,  1.06it/s]

...

Processing Companies: 100%|████████████▉| 10195/10197 [2:25:09<00:02,  1.03s/it]

Fetching 10-K links for LIGAND PHARMACEUTICALS INC (CIK: 886163)...


Processing Companies: 100%|████████████▉| 10196/10197 [2:25:10<00:00,  1.01it/s]

Fetching 10-K links for LIGAND PHARMACEUTICALS INC (CIK: 886163)...


Processing Companies: 100%|█████████████| 10197/10197 [2:25:11<00:00,  1.17it/s]
